## foundations version of Aircraft sizing

In [1]:
from modeling.compute import Par, Var
from modeling.execution import Component
from foundations.functionalsets import ResidualSet
from foundations.functionalsets import EliminationSet
from foundations.functionalsets import EliminationKeepSet
from foundations.functionalsets import Functional
from foundations.functionalsets import DefaultResidualSolver
from foundations.functionalsets import FeedForwardSolver
from foundations.functionalsets import FunctionalComp

In [2]:
A = Var('A') # aspect ratio (unitless)
V = Var('V', unit='m/s')
S = Var('S', unit='m^2')
W = Var('W')
Ww = Var('W_w', unit='N', forceunit=True)

W0 = Par('W0', 4940, 'N')
N = Par('N', 2.5)
t = Par('t', 0.12, 'm')
rhoS = Par('rhoS', 45.42, 'N/m^2')

F1 = FunctionalComp(Component.fromsympy(
    rhoS*S+8.71e-5*N/t*A**(3/2)*S**(1/2)*(W0*W)**(1/2), Ww, 
    arg_mapping=True))
F2 = FunctionalComp(Component.fromsympy(
    W0+Ww, W, 
    arg_mapping=True))

StructSet = F1.projectable.merge(F2.projectable)#etc...
Struct = StructSet.project((A,V,S))
#Struct.solver = FeedForwardSolver([F1, F2])

In [34]:
y1 = {vr:vr.varval if vr.varval else 1.01 for vr in F1.projected}
y2 = {vr:vr.varval if vr.varval else 1.01 for vr in F2.projected}

In [35]:
F1.solve(y1),F2.solve(y2)

({W_w: DeviceArray(46.00495064, dtype=float64)},
 {W: DeviceArray(4941.01, dtype=float64)})

In [36]:
y3 = {vr:vr.varval if vr.varval else 1.01 for vr in Struct.projected}

In [37]:
y3

{A: 1.01, V: 1.01, S: 1.01, W0: 4940, N: 2.5, t: 0.12, rhoS: 45.42}

In [39]:
z3 = Struct.solve(y3)

In [42]:
z3,F1.solve({**y1, **z3}),F2.solve({**y2, **z3})

({W: 4995.069245788502, W_w: 55.069245788501966},
 {W_w: DeviceArray(55.06924579, dtype=float64)},
 {W: DeviceArray(4995.06924579, dtype=float64)})